In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from IPython.display import clear_output
!pip install "/kaggle/input/isic2024-git/library/lightning-2.3.3-py3-none-any.whl"
!pip install "/kaggle/input/isic2024-git/library/yacs-0.1.8-py3-none-any.whl"
# clear_output()
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from io import BytesIO
import os
import sys
from torchvision import transforms
from torch.utils.data import Dataset
import torch._dynamo

torch._dynamo.config.suppress_errors = True

import timm
sys.path.append("/kaggle/input/isic2024-git")
import h5py
import io
from tqdm import tqdm
from classification import *
from lightning.pytorch import Trainer
from torch.utils.data import DataLoader

torch.set_float32_matmul_precision("high")
class ISIC_test_image(Dataset):
    def __init__(self, hdf5_path, metadata):
        self.hdf5_path = hdf5_path
        self.isic_ids = metadata["isic_id"].values

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        image_data = h5py.File(self.hdf5_path, mode="r")[self.isic_ids[idx]][()]
        image = Image.open(io.BytesIO(image_data)).resize((cfg.DATA.IMG_SIZE))
        image = transforms.ToTensor()(image)

        return image
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Processing /kaggle/input/isic2024-git/library/lightning-2.3.3-py3-none-any.whl
Processing /kaggle/input/isic2024-git/library/yacs-0.1.8-py3-none-any.whl


In [2]:
TEST_METADATA = "/kaggle/input/isic-2024-challenge/test-metadata.csv"
TEST_HDF5 = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
TEST_SUBMISSION = "/kaggle/input/isic-2024-challenge/sample_submission.csv"

In [3]:
test_metadata = pd.read_csv(TEST_METADATA)

test_loader = ISIC_test_image(TEST_HDF5, test_metadata)

test_dataset = DataLoader(
    test_loader,
    batch_size=16,
    num_workers=4,
    prefetch_factor=8,
    shuffle=False,
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = convnext_tiny(
    pretrained=False,
    in_22k=cfg.TRAIN.CONVEXT.IN22K,
    in_chans=cfg.DATA.IN_CHANNEL,
    num_classes=cfg.DATA.NUM_CLASS,
    drop_path_rate=cfg.TRAIN.CONVEXT.DROPOUT,
)

checkpoint_dict = {
    "fold1": "/kaggle/input/convnext_tiny/pytorch/default/1/ckpt_auc_0.1934-v4.ckpt",
    "fold2": "/kaggle/input/convnext_tiny/pytorch/default/1/ckpt_auc_0.1936-v2.ckpt",
    "fold3": "/kaggle/input/convnext_tiny/pytorch/default/1/ckpt_recall_0.9617.ckpt",
    "fold4": "/kaggle/input/convnext_tiny/pytorch/default/1/ckpt_recall_0.9671.ckpt",
    "fold5": "/kaggle/input/convnext_tiny/pytorch/default/1/ckpt_recall_0.9719.ckpt",
}
classifier = Classifier(
    model,
    cfg.DATA.CLASS_WEIGHT,
    cfg.DATA.NUM_CLASS,
    cfg.OPT.LEARNING_RATE,
    cfg.OPT.FACTOR_LR,
    cfg.OPT.PATIENCE_LR,
)
classifier = classifier.to(device)
sum_predictions = 0
test_predictions = []
classifier.eval()
with torch.inference_mode():
    for images in tqdm(test_dataset, desc="Prediction Loop"):
        images = images.to(device)
        preds = 0
        for fold in range(1, 6):
            classifier = Classifier.load_from_checkpoint(
                checkpoint_path=checkpoint_dict[f"fold{fold}"],
                model=model,
                class_weight=cfg.DATA.CLASS_WEIGHT,
                num_classes=cfg.DATA.NUM_CLASS,
                learning_rate=cfg.OPT.LEARNING_RATE,
                factor_lr=cfg.OPT.FACTOR_LR,
                patience_lr=cfg.OPT.PATIENCE_LR,
            )
            preds += classifier(images)
            
        preds /= 5
        test_predictions.append(preds.cpu().numpy()[:, 1])

test_predictions = np.concatenate(test_predictions)

Prediction Loop:   0%|          | 0/1 [00:00<?, ?it/s][2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING] WON'T CONVERT forward /kaggle/input/isic2024-git/classification/convNext_model.py line 110 
[2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING] due to: 
[2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING] Traceback (most recent call last):
[2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING]   File "/opt/conda/lib/python3.10/site-packages/torch/_inductor/scheduler.py", line 1630, in create_backend
[2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING]     raise RuntimeError(
[2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING] torch._dynamo.exc.BackendCompilerFailed: backend='inductor' raised:
[2024-08-11 09:47:05,761] torch._dynamo.convert_frame: [WARNING] RuntimeError: Found Tesla P100-PCIE-16GB which is too old to be supported by the triton GPU compiler, which is used as the backend. Triton only supports de

In [5]:
example = pd.read_csv(TEST_SUBMISSION)
example.target = test_predictions
example.to_csv("submission.csv", index=False)